# Energy Diversity Based Ensemble Selection


## 1. Imports and Setup

In [ ]:
import os
import sys
import timeit
import numpy as np

import torch
from itertools import combinations

sys.path.append("/home/myid/bs83243/mastersProject/EnsembleBench/EnsembleBench/frameworks")

from pytorchUtility import (
    calAccuracy,
    calAveragePredictionVectorAccuracy,
    filterModelsFixed,
)

from EnsembleBench.groupMetrics import(
    calAllDiversityMetrics,
)

%load_ext autoreload
%autoreload 2

## 2. Configuration
Define dataset, models, paths, energy constraints, and other parameters here.

In [ ]:
# --- Dataset Configuration ---
DATASET = 'imagenet' 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PREDICTION_SUFFIX = '.pt'

PREDICTION_DIR = '/home/myid/bs83243/mastersProject/energy_constraint_ensemble/ValFileSorted/imagenet'
MODELS = np.array(['densenet201', 'resnet18', 'resnet152', 'resnext50', 'vgg19_bn', 
                       'efficientnet_b0', 'inception_v3', 'squeezeNet1_1', 'alexnet', 'vgg16'])
ENERGY_PROFILES = {
    0: 27.457, # densenet201
    1: 10.717, # resnet18
    2: 29.823, # resnet152
    3: 28.351, # resnext50
    4: 49.646, # vgg19_bn
    5: 8.794,  # efficientnet_b0
    6: 32.011, # inception_v3
    7: 10.646, # squeezeNet1_1
    8: 11.201, # alexnet
    9: 36.080  # vgg16
}

# --- Diversity Metrics ---
DIVERSITY_METRICS_LIST = ['CK', 'QS', 'BD', 'FK', 'KW', 'GD']

# --- Energy Constraint ---
TOTAL_ENERGY = sum(ENERGY_PROFILES.values())
ENERGY_CONSTRAINT_PERCENT = 50 # Set desired percentage
ENERGY_CONSTRAINT = TOTAL_ENERGY * (ENERGY_CONSTRAINT_PERCENT / 100.0)

# --- Diversity Calculation Settings ---
CROSS_VALIDATION = True
CROSS_VALIDATION_TIMES = 3
N_RANDOM_SAMPLES = 1000

print(f"Configuration:")
print(f"  Dataset: {DATASET}")
print(f"  Models: {MODELS}")
print(f"  Prediction Dir: {PREDICTION_DIR}")
print(f"  Device: {DEVICE}")
print(f"  Energy Constraint: {ENERGY_CONSTRAINT:.3f} kWh ({ENERGY_CONSTRAINT_PERCENT}% of Total {TOTAL_ENERGY:.3f} kWh)")
print(f"  Diversity Metrics: {DIVERSITY_METRICS_LIST}")

## 3. Helper Functions

In [ ]:
def analyze_team_accuracies(team_list, team_accuracy_dict, description="Analysis"):
    """Analyzes and prints accuracy statistics for a list of teams."""
    if not team_list:
        print(f"{description}: No teams provided for analysis.")
        return None, 0.0, 0.0, 0.0, 0.0
        
    # Create a dictionary mapping the team to its accuracy for the provided list
    filtered_accuracy_mapping = {}
    for team in team_list:
        team_key = ''.join(map(str, team))
        if team_key in team_accuracy_dict:
             filtered_accuracy_mapping[team_key] = team_accuracy_dict[team_key]
        else:
            print(f"Warning: Team {team_key} not found in team_accuracy_dict.")
            
    if not filtered_accuracy_mapping:
        print(f"{description}: No valid teams found in accuracy dictionary.")
        return None, 0.0, 0.0, 0.0, 0.0

    accuracies = list(filtered_accuracy_mapping.values())
    max_team_key = max(filtered_accuracy_mapping, key=filtered_accuracy_mapping.get)
    min_team_key = min(filtered_accuracy_mapping, key=filtered_accuracy_mapping.get)
    max_accuracy = filtered_accuracy_mapping[max_team_key]
    min_accuracy = filtered_accuracy_mapping[min_team_key]
    mean_accuracy = np.mean(accuracies)
    median_accuracy = np.median(accuracies)

    print(f"--- {description} --- ({len(team_list)} teams)")
    print(f"  Max Accuracy Team: {max_team_key} with Accuracy: {max_accuracy:.3f}")
    print(f"  Min Accuracy Team: {min_team_key} with Accuracy: {min_accuracy:.3f}")
    print(f"  Mean Accuracy: {mean_accuracy:.3f}")
    print(f"  Median Accuracy: {median_accuracy:.3f}")
    print(f"-----------------------")
    return max_team_key, max_accuracy, min_accuracy, mean_accuracy, median_accuracy

def is_team_within_energy_constraint(team, energy_profile_dict, energy_constraint):
    """Checks if a team's energy consumption is within the constraint."""
    try:
        total_energy = sum(energy_profile_dict[model_idx] for model_idx in team)
        return total_energy <= energy_constraint
    except KeyError as e:
        print(f"Error: Model index {e} not found in energy profiles.")
        return False

## 4. Core Logic Functions

In [ ]:
def load_predictions(prediction_dir, models, suffix, device):
    """Loads prediction and label vectors for given models."""
    print("Loading predictions...")
    labelVectorsList = []
    predictionVectorsList = []
    individualAccuracies = []
    model_load_times = {}
    
    start_total_time = timeit.default_timer()
    for i, m in enumerate(models):
        start_model_time = timeit.default_timer()
        predictionPath = os.path.join(prediction_dir, m + suffix)
        try:
            prediction = torch.load(predictionPath, map_location=device)
            predictionVectors = prediction['predictionVectors']
            labelVectors = prediction['labelVectors']
            
            # Apply softmax and move to CPU
            predictionVectorsList.append(torch.nn.functional.softmax(predictionVectors, dim=-1).cpu())
            labelVectorsList.append(labelVectors.cpu())
            
            # Calculate individual accuracy
            acc = calAccuracy(predictionVectors, labelVectors)[0].cpu().item()
            individualAccuracies.append(acc)
            
            end_model_time = timeit.default_timer()
            model_load_times[m] = end_model_time - start_model_time
            print(f"  Loaded {m}: Accuracy={acc:.4f}, Time={model_load_times[m]:.2f}s")
            
        except FileNotFoundError:
            print(f"Error: Prediction file not found at {predictionPath}")
            # Handle error appropriately - skip model, raise exception, etc.
            return None, None, None, None # Indicate failure
        except KeyError as e:
            print(f"Error: Key {e} not found in prediction file {predictionPath}")
            return None, None, None, None
            
    end_total_time = timeit.default_timer()
    print(f"Finished loading predictions. Total time: {end_total_time - start_total_time:.2f}s")
    
    if not labelVectorsList:
        print("Error: No predictions were loaded successfully.")
        return None, None, None, None
        
    # Basic validation
    print(f"\nLoaded {len(predictionVectorsList)} models.")
    print(f"  Prediction vector size (first model): {predictionVectorsList[0].size()}")
    print(f"  Label vector size: {labelVectorsList[0].size()}")
    print(f"  Individual Accuracies: {[f'{acc:.4f}' for acc in individualAccuracies]}")
    print(f"  Min/Avg/Max Accuracy: {np.min(individualAccuracies):.4f} / {np.mean(individualAccuracies):.4f} / {np.max(individualAccuracies):.4f}")
    
    return predictionVectorsList, labelVectorsList[0], individualAccuracies, model_load_times

In [ ]:
def calculate_team_accuracies(prediction_vectors_list, label_vectors, num_models):
    """Calculates accuracy for all possible team combinations (soft voting)."""
    print("\nCalculating team accuracies (soft voting)...")
    teamAccuracyDict = {}
    all_teams = []
    startTime = timeit.default_timer()
    
    for n in range(2, num_models + 1):
        comb = combinations(list(range(num_models)), n)
        count = 0
        for selectedModels in list(comb):
            tmpAccuracy = calAveragePredictionVectorAccuracy(prediction_vectors_list, label_vectors, modelsList=selectedModels)[0].cpu().item()
            teamName = "".join(map(str, selectedModels))
            teamAccuracyDict[teamName] = tmpAccuracy
            all_teams.append(selectedModels) # Store the tuple of indices
            count += 1
        print(f"  Calculated accuracy for {count} teams of size {n}")
            
    endTime = timeit.default_timer()
    print(f"Finished calculating team accuracies. Total time: {endTime - startTime:.2f}s")
    print(f"  Total number of teams (size >= 2): {len(teamAccuracyDict)}")
    # analyze_team_accuracies(all_teams, teamAccuracyDict, "Overall Team Accuracy Analysis") # Optional: Analyze here
    return teamAccuracyDict, all_teams

In [ ]:
def calculate_disagreement_samples(prediction_vectors_list, label_vectors):
    """Identifies samples where models disagree on the prediction."""
    print("\nCalculating disagreement samples...")
    startTime = timeit.default_timer()
    
    target = label_vectors # Assuming label_vectors is already the target tensor
    batchSize = target.size(0)
    predictionList = []
    
    # Get predicted class index for each model
    for pVL in prediction_vectors_list:
        _, pred = pVL.max(dim=1)
        predictionList.append(pred)
        
    sampleID = []
    sampleTarget = []
    predictions = []
    predVectors = []
    disagreement_count = 0
    
    for i in range(batchSize):
        model_preds_for_sample = [p[i].item() for p in predictionList]
        # Check if all predictions are the same
        if len(set(model_preds_for_sample)) > 1:
            disagreement_count += 1
            sampleID.append(i)
            sampleTarget.append(target[i].item())
            predictions.append(model_preds_for_sample)
            # Store prediction vectors (probabilities) for disagreed samples
            predVectors.append([pv[i].numpy() for pv in prediction_vectors_list]) # Convert to numpy here if needed later
            
    endTime = timeit.default_timer()
    print(f"Finished calculating disagreement samples. Total time: {endTime - startTime:.2f}s")
    print(f"  Found {disagreement_count} disagreement samples out of {batchSize}.")
    
    # Convert lists to numpy arrays
    sampleID = np.array(sampleID)
    sampleTarget = np.array(sampleTarget)
    predictions = np.array(predictions)
    # Ensure predVectors is a NumPy array of the correct shape if needed by downstream functions
    # This conversion might be memory-intensive for large datasets/vectors
    try:
        predVectors = np.array(predVectors) 
    except ValueError as e:
         print(f"Warning: Could not convert predVectors to a uniform NumPy array: {e}. Keeping as list of lists.")
         # Handle non-uniform shapes if necessary
         pass 

    return sampleID, sampleTarget, predictions, predVectors

In [ ]:
def calculate_diversity_scores(all_teams, sampleID, sampleTarget, predictions, predVectors, 
                               diversity_metrics_list, cross_validation=True, 
                               cross_validation_times=3, n_random_samples=1000):
    """Calculates diversity scores for all provided teams."""
    print("\nCalculating diversity scores...")
    np.random.seed(0) # for reproducibility
    diversityScoresList = []
    teamSizeList = []
    teamListProcessed = [] # Keep track of teams for which scores are calculated
    nModels = predictions.shape[1] # Total number of base models
    modelIdx = list(range(nModels))
    
    startTime = timeit.default_timer()
    
    processed_count = 0
    for selectedModels in all_teams: # Iterate through the teams generated earlier
        n = len(selectedModels)
        # Filter disagreement data for the current team
        # Note: filterModelsFixed expects predictions/predVectors for *all* models
        teamSampleID, teamSampleTarget, teamPredictions, teamPredVectors = filterModelsFixed(
            sampleID, sampleTarget, predictions, predVectors, selectedModels
        )
        
        if len(teamPredictions) == 0:
            print(f"Warning: No disagreement samples found for team {selectedModels}. Skipping diversity calculation.")
            continue # Skip this team if no disagreements
        
        if cross_validation:
            if len(teamPredictions) < n_random_samples:
                # print(f"Warning: Team {selectedModels} has fewer samples ({len(teamPredictions)}) than n_random_samples ({n_random_samples}). Using all samples.")
                current_n_random_samples = len(teamPredictions)
            else:
                current_n_random_samples = n_random_samples
                
            tmpMetrics = []
            for _ in range(cross_validation_times):
                if current_n_random_samples > 0:
                    randomIdx = np.random.choice(np.arange(teamPredictions.shape[0]), current_n_random_samples, replace=False)
                    tmpMetrics.append(calAllDiversityMetrics(teamPredictions[randomIdx], teamSampleTarget[randomIdx], diversity_metrics_list))
                else:
                     tmpMetrics.append([np.nan] * len(diversity_metrics_list)) # Handle case with 0 samples
            # Handle potential NaNs if a metric calculation failed
            tmpMetrics = np.nanmean(np.array(tmpMetrics), axis=0) 
        else:
            tmpMetrics = np.array(calAllDiversityMetrics(teamPredictions, teamSampleTarget, diversity_metrics_list))
        
        diversityScoresList.append(tmpMetrics)
        teamSizeList.append(n)
        teamListProcessed.append(selectedModels)
        processed_count += 1
        if processed_count % 100 == 0:
             print(f"  Processed diversity for {processed_count}/{len(all_teams)} teams...")
            
    endTime = timeit.default_timer()
    print(f"Finished calculating diversity scores. Total time: {endTime - startTime:.2f}s")
    print(f"  Calculated scores for {len(teamListProcessed)} teams.")
    
    diversityScoresList = np.array(diversityScoresList)
    teamSizeList = np.array(teamSizeList)
    teamListProcessed = np.array(teamListProcessed, dtype=object)
    
    return diversityScoresList, teamSizeList, teamListProcessed

In [ ]:
def create_team_diversity_dict(team_list, diversity_scores_list):
    """Creates a dictionary mapping team string to its diversity scores array."""
    team_diversity_dict = {}
    if len(team_list) != len(diversity_scores_list):
        print("Error: team_list and diversity_scores_list must have the same length.")
        return None
    for i, team in enumerate(team_list):
        team_key = ''.join(map(str, team))
        team_diversity_dict[team_key] = diversity_scores_list[i]
    print(f"\nCreated team-to-diversity dictionary with {len(team_diversity_dict)} entries.")
    return team_diversity_dict

In [ ]:
def exhaustive_search_constrained(all_teams, team_accuracy_dict, energy_profile_dict, energy_constraint):
    """Performs exhaustive search for the best team under energy constraints."""
    print(f"\nPerforming Exhaustive Search with Energy Constraint <= {energy_constraint:.3f} kWh...")
    startTime = timeit.default_timer()
    
    filtered_team_list = []
    for team in all_teams:
        if is_team_within_energy_constraint(team, energy_profile_dict, energy_constraint):
            filtered_team_list.append(team)
            
    endTime = timeit.default_timer()
    print(f"Finished filtering teams. Time: {endTime - startTime:.2f}s")
    print(f"  Found {len(filtered_team_list)} teams within the energy constraint.")
    
    # Analyze the filtered teams
    best_team_key, max_acc, _, _, _ = analyze_team_accuracies(
        filtered_team_list, 
        team_accuracy_dict, 
        description=f"Exhaustive Search (Constraint: {energy_constraint:.3f} kWh)"
    )
    
    return filtered_team_list, best_team_key, max_acc

In [ ]:
# Note: This is just for testing to find the max diversity
def greedy_search_max_diversity(all_teams, team_size_list, team_diversity_dict, energy_profile_dict, 
                                energy_constraint, diversity_metric_index, diversity_metrics_list):
    """Performs greedy search selecting teams by maximizing (or minimizing) a specific diversity metric."""
    metric_name = diversity_metrics_list[diversity_metric_index]
    print(f"\nPerforming Greedy Search by MINIMIZING '{metric_name}' (Index {diversity_metric_index}) with Energy Constraint <= {energy_constraint:.3f} kWh...")
    startTime = timeit.default_timer()
    
    max_diversity_filtered_team_list = []
    num_models = max(team_size_list) if team_size_list.size > 0 else 0

    # Map team sizes to lists of (team, diversity_score)
    teams_by_size = {size: [] for size in range(2, num_models + 1)}
    for team, size in zip(all_teams, team_size_list):
         team_key = ''.join(map(str, team))
         if team_key in team_diversity_dict:
             diversity_score = team_diversity_dict[team_key][diversity_metric_index]
             teams_by_size[size].append((team, diversity_score))

    current_best_team = None
    for i in range(2, num_models + 1):
        potential_teams_this_size = []
        # Filter teams of current size that satisfy energy constraint and contain previous best team (if any)
        for team, diversity_score in teams_by_size[i]:
            if is_team_within_energy_constraint(team, energy_profile_dict, energy_constraint):
                if current_best_team is None or set(current_best_team).issubset(set(team)):
                     potential_teams_this_size.append((team, diversity_score))
        
        if not potential_teams_this_size:
            # print(f"  No suitable teams found for size {i}. Stopping greedy search.")
            break # Stop if no valid team can be found for this size
            
        # Find the team with the minimum diversity score among potential teams
        best_team_this_size = min(potential_teams_this_size, key=lambda x: x[1] if not np.isnan(x[1]) else float('inf'))
        
        # Check if a valid team was found (diversity score is not inf)
        if not np.isinf(best_team_this_size[1]):
             current_best_team = best_team_this_size[0]
             max_diversity_filtered_team_list.append(current_best_team)
             
    endTime = timeit.default_timer()
    print(f"Finished greedy search. Time: {endTime - startTime:.2f}s")
    print(f"  Selected {len(max_diversity_filtered_team_list)} teams.")
    
    return max_diversity_filtered_team_list

In [ ]:
def calculate_mean_diversity_thresholds(diversity_scores_list, diversity_metrics_list):
    """Calculates the mean threshold for each diversity metric."""
    print("\nCalculating mean diversity thresholds...")
    thresholds = {}
    if diversity_scores_list.size == 0:
        print("  Warning: Diversity scores list is empty. Cannot calculate thresholds.")
        return {dm: np.nan for dm in diversity_metrics_list}
        
    for j, dm in enumerate(diversity_metrics_list):
        mean_value = np.nanmean(diversity_scores_list[..., j])
        thresholds[dm] = mean_value
        print(f"  Threshold for {dm}: {mean_value:.4f}")
    return thresholds

In [ ]:
def greedy_search_threshold(all_teams, team_size_list, team_diversity_dict, energy_profile_dict, 
                            energy_constraint, diversity_metric_index, thresholds, diversity_metrics_list):
    """Performs greedy search selecting teams under a diversity threshold."""
    # NOTE: The diversity is normalized to have lower score with higher diversity
    metric_name = diversity_metrics_list[diversity_metric_index]
    threshold = thresholds.get(metric_name, float('inf')) # Get threshold for the specific metric
    print(f"\nPerforming Greedy Search using Threshold for '{metric_name}' (<= {threshold:.4f}) with Energy Constraint <= {energy_constraint:.3f} kWh...")
    startTime = timeit.default_timer()
    
    threshold_filtered_team_list = []
    num_models = max(team_size_list) if team_size_list.size > 0 else 0

    # Map team sizes to lists of (team, diversity_score)
    teams_by_size = {size: [] for size in range(2, num_models + 1)}
    for team, size in zip(all_teams, team_size_list):
         team_key = ''.join(map(str, team))
         if team_key in team_diversity_dict:
             diversity_score = team_diversity_dict[team_key][diversity_metric_index]
             teams_by_size[size].append((team, diversity_score))

    processed_teams_this_round = set()
    all_selected_teams = set()

    for i in range(2, num_models + 1):
        teams_under_threshold_this_size = []
        current_round_added_tuples = set()

        for team, diversity_score in teams_by_size[i]:
            team_tuple = tuple(sorted(team)) # Use sorted tuple for set operations
            # Check energy constraint, diversity threshold, and if it extends a previously selected team
            if is_team_within_energy_constraint(team, energy_profile_dict, energy_constraint) and \
               not np.isnan(diversity_score) and diversity_score <= threshold and \
               (i == 2 or any(set(prev_team).issubset(set(team)) for prev_team in processed_teams_this_round)): # Check against teams added in the *previous* size
                
                teams_under_threshold_this_size.append(team)
                current_round_added_tuples.add(team_tuple)
                all_selected_teams.add(team_tuple)
                
        # Update the set of teams added in this round for the next iteration's check
        processed_teams_this_round = {tuple(sorted(t)) for t in teams_under_threshold_this_size}
        # print(f"  Size {i}: Found {len(teams_under_threshold_this_size)} teams under threshold.")
        
        # Break if no teams are found for a size
        if not teams_under_threshold_this_size and i > 2:
            print(f"  No teams found for size {i} extending previous selections. Stopping.")
            break
            
    # Convert final set of tuples back to list of lists/tuples if needed
    threshold_filtered_team_list = [list(t) for t in all_selected_teams]
            
    endTime = timeit.default_timer()
    print(f"  Selected {len(threshold_filtered_team_list)} teams.")
    
    return threshold_filtered_team_list

## 5. Execution Flow

In [ ]:
# 1. Load Predictions and Calculate Individual Accuracies
predictionVectorsList, labelVectors, individualAccuracies, modelLoadTimes = load_predictions(
    PREDICTION_DIR, MODELS, PREDICTION_SUFFIX, DEVICE
)

# Check if loading was successful
if predictionVectorsList is None:
    raise RuntimeError("Failed to load prediction data. Please check paths and file contents.")

In [ ]:
# 2. Calculate Team Accuracies (Exhaustive Soft Voting)
teamAccuracyDict, allTeamsList = calculate_team_accuracies(
    predictionVectorsList, labelVectors, len(MODELS)
)

# Analyze overall team accuracies (optional)
analyze_team_accuracies(allTeamsList, teamAccuracyDict, "Overall Team Accuracy Analysis (All Teams >= 2)")

In [ ]:
# 3. Calculate Disagreement Samples
sampleID, sampleTarget, disagreePredictions, disagreePredVectors = calculate_disagreement_samples(
    predictionVectorsList, labelVectors
)

In [ ]:
# 4. Calculate Diversity Scores
diversityScoresList, teamSizeList, processedTeamList = calculate_diversity_scores(
    allTeamsList, 
    sampleID, 
    sampleTarget, 
    disagreePredictions, 
    disagreePredVectors, 
    DIVERSITY_METRICS_LIST,
    cross_validation=CROSS_VALIDATION,
    cross_validation_times=CROSS_VALIDATION_TIMES,
    n_random_samples=N_RANDOM_SAMPLES
)

# 5. Create Team-to-Diversity Dictionary
teamDiversityDict = create_team_diversity_dict(processedTeamList, diversityScoresList)

## 6. Ensemble Selection Strategies

In [ ]:
# 6a. Exhaustive Search with Energy Constraint
exhaustiveFilteredTeams, exhaustiveBestTeamKey, exhaustiveMaxAcc = exhaustive_search_constrained(
    allTeamsList, # Use the full list of teams
    teamAccuracyDict,
    ENERGY_PROFILES,
    ENERGY_CONSTRAINT
)

In [ ]:
# 6b. Greedy Search by Maximizing/Minimizing Diversity with Energy Constraint
# Ensure teamDiversityDict is not None before proceeding
for GREEDY_DIVERSITY_METRIC_INDEX, diversity_metric in enumerate(DIVERSITY_METRICS_LIST):
    greedyMaxDivTeams = greedy_search_max_diversity(
        processedTeamList, # Use the list corresponding to diversity scores
        teamSizeList, 
        teamDiversityDict,
        ENERGY_PROFILES,
        ENERGY_CONSTRAINT,
        GREEDY_DIVERSITY_METRIC_INDEX, # Index for the metric (e.g., KW)
        DIVERSITY_METRICS_LIST
    )
    
    # Analyze the results of the greedy search
    analyze_team_accuracies(
        greedyMaxDivTeams, 
        teamAccuracyDict, 
        description=f"Greedy Search (Minimizing {DIVERSITY_METRICS_LIST[GREEDY_DIVERSITY_METRIC_INDEX]}) Constrained"
    )
else:
    print("\nSkipping Greedy Max Diversity Search: teamDiversityDict not available.")

In [ ]:
# 6c. Greedy Search using Diversity Threshold with Energy Constraint

# First, calculate thresholds
meanThresholds = calculate_mean_diversity_thresholds(diversityScoresList, DIVERSITY_METRICS_LIST)

for GREEDY_DIVERSITY_METRIC_INDEX, diversity_metric in enumerate(DIVERSITY_METRICS_LIST):
    greedyThresholdTeams = greedy_search_threshold(
        processedTeamList, # Use the list corresponding to diversity scores
        teamSizeList,
        teamDiversityDict,
        ENERGY_PROFILES,
        ENERGY_CONSTRAINT,
        GREEDY_DIVERSITY_METRIC_INDEX, # Index for the metric (e.g., KW)
        meanThresholds, # Pass the calculated thresholds
        DIVERSITY_METRICS_LIST
    )
        
        # Analyze the results
    analyze_team_accuracies(
        greedyThresholdTeams, 
        teamAccuracyDict, 
        description=f"Greedy Search (Threshold {DIVERSITY_METRICS_LIST[GREEDY_DIVERSITY_METRIC_INDEX]} <= {meanThresholds.get(DIVERSITY_METRICS_LIST[GREEDY_DIVERSITY_METRIC_INDEX], 'N/A'):.4f}) Constrained"
    )